In [1]:
!pip install ../gentrain/.

ERROR: Invalid requirement: '../gentrain/.': Expected package name at the start of dependency specifier
    ../gentrain/.
    ^
Hint: It looks like a path. File '../gentrain/.' does not exist.


In [2]:
import pandas as pd
import plotly.express as px
from gentrain.evaluation import get_infection_rate

## Load Nextclade Results fors RKI Sequences
SARS-CoV-2 Sequenzdaten aus Deutschland: https://zenodo.org/records/14810854

Dataset: https://github.com/robert-koch-institut/SARS-CoV-2-Sequenzdaten_aus_Deutschland

In [3]:
nextclade_results_df = pd.read_csv("../../data/rki_sars_cov_2.csv", delimiter=";", low_memory=False)

Only keep relevant columns.

In [4]:
nextclade_clean_df = nextclade_results_df[
    ["seqName", "substitutions", "insertions", "deletions", "missing", "nonACGTNs", "totalSubstitutions",
     "totalInsertions", "totalDeletions", "totalMissing", "totalNonACGTNs", "alignmentScore", "alignmentStart",
     "alignmentEnd", "coverage", "clade", "Nextclade_pango", "clade_who", "qc.overallScore"]]

In [5]:
nextclade_clean_df.head()

,seqName,substitutions,insertions,deletions,missing,nonACGTNs,totalSubstitutions,totalInsertions,totalDeletions,totalMissing,totalNonACGTNs,alignmentScore,alignmentStart,alignmentEnd,coverage,clade,Nextclade_pango,clade_who,qc.overallScore
0,IMS-10087-CVDP-050EE515-A324-43DF-8E4E-0CFC0EE...,"C241T,C1912T,C2470T,A2832G,C3037T,T5386G,C6633...",28879:A,"6513-6515,11283-11287,21765-21770,21987-21995,...","1-54,11288-11291,28370,28884,29837-29903","S:8354,K:22882,R:22898,Y:25000,R:26530,S:26577",57.0,1.0,34.0,127.0,6.0,89210.0,1.0,29903.0,0.995552,21K,BA.1.1.14,Omicron,50.062500
1,IMS-10087-CVDP-051B8C1C-3D48-4E56-8BEC-3FDDD12...,"C241T,A2832G,C3037T,C3874T,T5386G,C5730T,C6968...",28879:A,"6513-6515,11283-11287,21765-21770,21987-21995,...","1-54,11288-11291,28370,28884,29837-29903","Y:5221,S:8354,Y:13620,Y:23248,Y:25000,R:26530,...",56.0,1.0,34.0,127.0,7.0,89214.0,1.0,29903.0,0.995519,21K,BA.1.18,Omicron,53.340278
2,IMS-10087-CVDP-0016B98F-5D26-4455-BECE-A3D66EF...,"C241T,T670G,C2790T,C3037T,G4184A,C4321T,C9344T...",28879:A,"11288-11296,21633-21641,28362-28369","1-11,28370,28884,29734-29759,29837-29903","Y:44,S:8354,K:22882,Y:25000,S:26577,Y:26858,Y:...",63.0,1.0,26.0,106.0,7.0,89249.0,1.0,29903.0,0.996221,21L,BA.2,Omicron,49.173611
3,IMS-10087-CVDP-00D244D8-C9E8-4FDD-A93E-D176E1A...,"C241T,T670G,C2790T,C3037T,G4184A,C9344T,A9424G...",28879:A,"11288-11296,21633-21641,28362-28369","1-54,28370,28884,29734-29759,29837-29870,29899...","Y:4321,K:22882,Y:25000,S:26577,Y:26858",62.0,1.0,26.0,121.0,5.0,89238.0,1.0,29903.0,0.995786,21L,BA.2.9,Omicron,25.000000
4,IMS-10087-CVDP-01A2C74B-54A8-47B1-B7E4-6562C62...,"C241T,A2832G,C3037T,T5386G,C5672T,G5924A,C9344...",28879:A,"11288-11296,21633-21641,28362-28369","1-54,28370,28884,29459-29484,29510,29734-29759...","K:22882,Y:25000,Y:25889,S:26577",63.0,1.0,26.0,142.0,4.0,89213.0,1.0,29903.0,0.995118,recombinant,XG,recombinant,16.000000


Only keep igs_id (drop other parts of fasta headers).

In [6]:
nextclade_clean_df.loc[:, "igs_id"] = nextclade_clean_df["seqName"].str.split(' ').str[0]
nextclade_clean_df = nextclade_clean_df.drop("seqName", axis=1)

/var/folders/2h/923cq6912sqb0snfvqqfdnmm0000gn/T/ipykernel_69646/1844192000.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nextclade_clean_df.loc[:, "igs_id"] = nextclade_clean_df["seqName"].str.split(' ').str[0]


In [7]:
print(f"{len(nextclade_clean_df)} sequences in the dataset.")

1238401 sequences in the dataset.


In [8]:
nextclade_clean_df.describe().apply(lambda s: s.apply('{0:.5f}'.format)).iloc[1:3]

,totalSubstitutions,totalInsertions,totalDeletions,totalMissing,totalNonACGTNs,alignmentScore,alignmentStart,alignmentEnd,coverage,qc.overallScore
mean,55.72137,2.79253,35.63678,367.98367,1.74396,88847.99312,20.99750,29878.08017,0.98613,87.35378
std,18.75749,19.73173,51.32371,403.05033,6.97231,513.63185,51.79426,46.19538,0.01358,4012.08991


## Load Metadata

In [9]:
source = "../../data/metadata.tsv"

In [10]:
with open(source) as csvfile:
    rki_metadata = pd.read_csv(csvfile, delimiter="\t", index_col=0)

In [11]:
rki_metadata

,date_of_sampling,sequencing_platform,sequencing_reason,isolation_source,lab_sequence_id,date_of_submission,version,prime_diagnostic_lab.demis_lab_id,prime_diagnostic_lab.postal_code,sequencing_lab.demis_lab_id,sequencing_lab.postal_code,lineages
igs_id,,,,,,,,,,,,
IMS-10087-CVDP-0016B98F-5D26-4455-BECE-A3D66EF7EE1B,2022-03-14T00:00:00,ILLUMINA,random,Nasopharyngeal swab (specimen),2713fb003a41ed7367db1a884a2a6b3b67c520cb1b69f2...,2022-03-21T19:00:03,0,DEMIS-10087,50858.0,DEMIS-10087,50858.0,"[{""lineage"": ""BA.2"", ""method"": ""PANGOLIN_LATES..."
IMS-10087-CVDP-00D244D8-C9E8-4FDD-A93E-D176E1AD8CF0,2022-03-14T00:00:00,ILLUMINA,random,Nasopharyngeal swab (specimen),873a7cc28d29e3f17b0544ea6e9e8436defe32f6d60649...,2022-03-21T19:00:03,0,DEMIS-10087,50858.0,DEMIS-10087,50858.0,"[{""lineage"": ""BA.2.9"", ""method"": ""PANGOLIN_LAT..."
IMS-10087-CVDP-015FAF63-6C50-4C80-B3E2-E21BBE60B0DB,2022-03-14T00:00:00,ILLUMINA,random,Nasopharyngeal swab (specimen),05ab0bff57f513826af23e6437f66d6e469502d0d49bbd...,2022-03-21T19:00:03,0,DEMIS-10087,50858.0,DEMIS-10087,50858.0,"[{""lineage"": ""BA.2"", ""method"": ""PANGOLIN_LATES..."
IMS-10087-CVDP-01A2C74B-54A8-47B1-B7E4-6562C623CEA0,2022-03-14T00:00:00,ILLUMINA,random,Nasopharyngeal swab (specimen),603656d9506d653092d8868ab793c1a3aaf800c8f1302c...,2022-03-21T19:00:03,0,DEMIS-10087,50858.0,DEMIS-10087,50858.0,"[{""lineage"": ""XG"", ""method"": ""PANGOLIN_LATEST""..."
IMS-10087-CVDP-01F3AE15-DBF4-410E-B1C6-1D491D0B2974,2022-03-14T00:00:00,ILLUMINA,random,Nasopharyngeal swab (specimen),ccec324839abf75b0351eda586a6662e58246050f8d829...,2022-03-21T19:00:03,0,DEMIS-10087,50858.0,DEMIS-10087,50858.0,"[{""lineage"": ""BA.2"", ""method"": ""PANGOLIN_LATES..."
...,...,...,...,...,...,...,...,...,...,...,...,...
IGS-10285-CVDP-C0713711-FC9B-478D-831B-6D48A55898E2,2025-04-16T00:00:00,OXFORD_NANOPORE,clinical,NaN,IMSSC2-4-2025-2506803,2025-05-12T00:00:00,0,DEMIS-10285,13353.0,DEMIS-10285,13353.0,"[{""lineage"": ""LF.7"", ""method"": ""PANGOLIN_LATES..."
IGS-10285-CVDP-C8E27DB9-5178-4840-9D81-801E926C3F01,2025-04-22T00:00:00,OXFORD_NANOPORE,random,NaN,IMSSC2-87-2025-00023,2025-05-12T00:00:00,0,DEMIS-10032,14197.0,DEMIS-10285,13353.0,"[{""lineage"": ""XEC.8"", ""method"": ""PANGOLIN_LATE..."
IGS-10285-CVDP-D607260C-41BA-494B-86C3-545E812DEB7A,2025-04-22T00:00:00,OXFORD_NANOPORE,random,NaN,IMSSC2-100-2025-00042,2025-05-12T00:00:00,0,DEMIS-10184,24106.0,DEMIS-10285,13353.0,"[{""lineage"": ""XEC.2.3"", ""method"": ""PANGOLIN_LA..."


## Merge RKI metadata with Nextclade Results

In [12]:
merged_df = pd.merge(nextclade_clean_df, rki_metadata, on='igs_id', how='outer')
igs_id_column = merged_df.pop('igs_id')
merged_df.insert(0, 'igs_id', igs_id_column)

In [13]:
merged_df.head()

,igs_id,substitutions,insertions,deletions,missing,nonACGTNs,totalSubstitutions,totalInsertions,totalDeletions,totalMissing,...,sequencing_reason,isolation_source,lab_sequence_id,date_of_submission,version,prime_diagnostic_lab.demis_lab_id,prime_diagnostic_lab.postal_code,sequencing_lab.demis_lab_id,sequencing_lab.postal_code,lineages
0,IGS-10285-CVDP-000A3554-D2E6-49D7-9453-0519115...,"C241T,T670G,C897A,C2790T,C3037T,G3431T,T3565C,...",21608:TCATGCCGCTGT,"11288-11296,21633-21641,21652-21654,21765-2177...","1-78,23005,29841-29903",NaN,130.0,12.0,71.0,142.0,...,random,NaN,IMSSC2-304-2024-00095,2024-10-24T00:00:00,0,DEMIS-10543,49074.0,DEMIS-10285,13353.0,"[{""lineage"": ""KP.3.1.1"", ""method"": ""PANGOLIN_L..."
1,IGS-10285-CVDP-00280EAD-9CB1-4705-AB3A-7CBE55F...,"C241T,T670G,C897A,C1473T,C2644T,C2790T,C3037T,...",21608:TCATGCCGCTGT,"11288-11296,21633-21641,21652-21654,21765-2177...","1-78,21987,29841-29903",NaN,123.0,12.0,71.0,142.0,...,random,NaN,IMSSC2-4-2024-2410891,2024-10-17T00:00:00,0,DEMIS-10285,13353.0,DEMIS-10285,13353.0,"[{""lineage"": ""LB.1"", ""method"": ""PANGOLIN_LATES..."
2,IGS-10285-CVDP-0048F2F5-C83A-4383-9103-E0ECF5F...,"C241T,T670G,C897A,C2790T,C3037T,G3431T,T3565C,...",21608:TCATGCCGCTGT,"11288-11296,21633-21641,21652-21654,21765-2177...","1-78,29841-29903",NaN,133.0,12.0,71.0,141.0,...,random,NaN,IMSSC2-206-2024-00071,2024-10-24T00:00:00,0,DEMIS-10324,49124.0,DEMIS-10285,13353.0,"[{""lineage"": ""MC.25"", ""method"": ""PANGOLIN_LATE..."
3,IGS-10285-CVDP-006D7DAE-B90A-4060-B831-0D5C2E6...,"C241T,T670G,C897A,C2790T,C3037T,A3120G,G3431T,...",21608:TCATGCCGCTGT,"11288-11296,21633-21641,21652-21654,21765-2177...","1-78,11083,21987,29841-29903",NaN,131.0,12.0,71.0,143.0,...,random,NaN,IMSSC2-80-2024-00247,2024-12-18T00:00:00,0,DEMIS-10077,41169.0,DEMIS-10285,13353.0,"[{""lineage"": ""MC.24"", ""method"": ""PANGOLIN_LATE..."
4,IGS-10285-CVDP-00B3E1FA-F122-44FC-9563-3E93021...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,random,NaN,IMSSC2-159-2025-00097,2025-03-07T00:00:00,1,DEMIS-10145,88212.0,DEMIS-10285,13353.0,"[{""lineage"": ""XEC.18"", ""method"": ""PANGOLIN_LAT..."


## Data Overview

Get the sampling month for each sample.

In [14]:
import re
import seaborn as sns
from matplotlib import pyplot as plt

In [15]:
def retrieve_month_of_sampling(date):
    if type(date) != str:
        return None
    pattern = r"\d{4}-\d{2}"
    match = re.search(pattern, date)
    if match:
        return match.group()


merged_df["month_of_sampling"] = merged_df["date_of_sampling"].apply(lambda x: retrieve_month_of_sampling(x))

In [16]:
merged_df.columns

Index(['igs_id', 'substitutions', 'insertions', 'deletions', 'missing',
       'nonACGTNs', 'totalSubstitutions', 'totalInsertions', 'totalDeletions',
       'totalMissing', 'totalNonACGTNs', 'alignmentScore', 'alignmentStart',
       'alignmentEnd', 'coverage', 'clade', 'Nextclade_pango', 'clade_who',
       'qc.overallScore', 'date_of_sampling', 'sequencing_platform',
       'sequencing_reason', 'isolation_source', 'lab_sequence_id',
       'date_of_submission', 'version', 'prime_diagnostic_lab.demis_lab_id',
       'prime_diagnostic_lab.postal_code', 'sequencing_lab.demis_lab_id',
       'sequencing_lab.postal_code', 'lineages', 'month_of_sampling'],
      dtype='object')

## Add geo data

In [17]:
with open("../../data/DE_zip_codes.txt") as zip_codes_file:
    coords = {}
    cities = {}
    states = {}
    for line in zip_codes_file:
        values = line.split('\t')
        coords[values[1]] = (values[9], values[10])
        cities[values[1]] = values[7]
        states[values[1]] = values[3]

In [18]:
merged_df.dropna(subset=["sequencing_lab.postal_code"], inplace=True)
merged_df.dropna(subset=["prime_diagnostic_lab.postal_code"], inplace=True)

In [19]:
merged_df["sequencing_lab.postal_code"] = merged_df["sequencing_lab.postal_code"].apply(
    lambda x: str(f"0{int(x)}") if len(str(int(x))) == 4 else str(int(x)))
merged_df["prime_diagnostic_lab.postal_code"] = merged_df["prime_diagnostic_lab.postal_code"].apply(
    lambda x: str(f"0{int(x)}") if len(str(int(x))) == 4 else str(int(x)))
merged_df["prime_diagnostic_lab.state"] = merged_df["prime_diagnostic_lab.postal_code"].apply(lambda x: states[x] if x in states else None)
merged_df["prime_diagnostic_lab.city"] = merged_df["prime_diagnostic_lab.postal_code"].apply(lambda x: cities[x] if x in states else None)

In [20]:
columns = list(merged_df)
columns.insert(columns.index("prime_diagnostic_lab.postal_code"), columns.pop(columns.index("prime_diagnostic_lab.state")))
columns.insert(columns.index("prime_diagnostic_lab.postal_code"), columns.pop(columns.index("prime_diagnostic_lab.city")))
merged_df = merged_df.loc[:, columns]

## Dataset Extraction

Exclude high substitution sequences from Offenbach.

In [21]:
merged_df = merged_df[merged_df["totalSubstitutions"] < 300]

Exclude sequences outside of obligatory sequencing period.

In [22]:
average_distance_calculation_runtime = (330*60)/((5000*(5000-1))/2)

In [23]:
merged_df[(merged_df['prime_diagnostic_lab.state'] == "Nordrhein-Westfalen")&(merged_df['date_of_sampling'] >= '2021-01-01')&(merged_df['date_of_sampling'] <= '2021-12-31')]

,igs_id,substitutions,insertions,deletions,missing,nonACGTNs,totalSubstitutions,totalInsertions,totalDeletions,totalMissing,...,date_of_submission,version,prime_diagnostic_lab.demis_lab_id,prime_diagnostic_lab.state,prime_diagnostic_lab.city,prime_diagnostic_lab.postal_code,sequencing_lab.demis_lab_id,sequencing_lab.postal_code,lineages,month_of_sampling
56620,IMS-10013-CVDP-0008D19D-85CE-4D61-85AD-6A55368...,"G210T,C241T,C745T,C3037T,G4181T,C6255T,C6402T,...",NaN,"22029-22034,28248-28253,28271","1-2,29867-29868","M:1573,Y:4165,R:24410,M:25601,Y:27143,R:28461",47.0,0.0,13.0,4.0,...,2022-01-07T19:00:03,0,DEMIS-10078,Nordrhein-Westfalen,"Bochum, Stadt",44799,DEMIS-10013,04779,"[{""lineage"": ""AY.125"", ""method"": ""PANGOLIN_LAT...",2021-12
56664,IMS-10013-CVDP-0011538C-EF82-4535-9ACA-4FC4FF1...,"G210T,C241T,G376T,C3037T,G4181T,C6402T,C7124T,...",NaN,"22029-22034,28248-28253,28271",1-29,"M:1573,R:12785,R:24410,M:25601,R:28461",46.0,0.0,13.0,29.0,...,2022-01-17T19:00:05,0,DEMIS-10078,Nordrhein-Westfalen,"Bochum, Stadt",44799,DEMIS-10013,04779,"[{""lineage"": ""AY.121"", ""method"": ""PANGOLIN_LAT...",2021-12
56872,IMS-10013-CVDP-00273DDA-22E4-4FD9-881F-2A02860...,"G210T,C241T,C745T,C1093T,C3037T,G4181T,C6402T,...",NaN,"7067,22029-22034,28248-28253,28271",1-30,"R:24410,K:25687,R:28461,W:29867",41.0,0.0,14.0,30.0,...,2021-08-27T18:00:04,0,DEMIS-10078,Nordrhein-Westfalen,"Bochum, Stadt",44799,DEMIS-10013,04779,"[{""lineage"": ""AY.125"", ""method"": ""PANGOLIN_LAT...",2021-08
56909,IMS-10013-CVDP-002CBAE9-A99D-44C4-A497-AC492D9...,"C241T,C913T,T2899C,C3037T,C3267T,C5388A,C5986T...",NaN,"11288-11296,21765-21770,21991-21993,28271","1-30,16511-16748",Y:9857,32.0,0.0,19.0,268.0,...,2021-03-01T19:00:04,0,DEMIS-10078,Nordrhein-Westfalen,"Bochum, Stadt",44799,DEMIS-10013,04779,"[{""lineage"": ""B.1.1.7"", ""method"": ""PANGOLIN_LA...",2021-02
57469,IMS-10013-CVDP-00706,"G204T,C241T,T445C,C3037T,C6040T,C6286T,C6408T,...",NaN,2567-2572,"1-9,2269-2504",NaN,22.0,0.0,6.0,245.0,...,2021-02-07T00:00:00,0,DEMIS-10078,Nordrhein-Westfalen,"Bochum, Stadt",44799,DEMIS-10013,04779,"[{""lineage"": ""B.1.177.86"", ""method"": ""PANGOLIN...",2021-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224386,IMS-10595-CVDP-00069,"C241T,C3037T,C4543T,G5629T,G9526T,C11497T,G134...",22836:N,NaN,"1-54,5980,6884-6969,6996-7035,7044,7046,9307-9...","S:7037,K:14831,M:21541,K:21542,W:21543,K:21544...",28.0,1.0,0.0,939.0,...,2021-02-15T00:00:00,0,DEMIS-10595,Nordrhein-Westfalen,"Münster, Stadt",48153,DEMIS-10595,48153,"[{""lineage"": ""B.1.160"", ""method"": ""PANGOLIN_LA...",2021-02
1224387,IMS-10595-CVDP-00070,"C241T,A291G,C2094T,C3037T,C3602T,C6941T,C7945T...",NaN,NaN,"1-47,5980,7044,7046,9629,20526,29722-29903","R:6894,K:6895,M:6896,K:6897,R:6898,W:6900",22.0,0.0,0.0,234.0,...,2021-02-15T00:00:00,0,DEMIS-10595,Nordrhein-Westfalen,"Münster, Stadt",48153,DEMIS-10595,48153,"[{""lineage"": ""B.1.221"", ""method"": ""PANGOLIN_LA...",2021-02
1229807,IMS-10640-CVDP-1BDE8D2C-6C37-4D7F-B37F-3DE4711...,"G204T,C241T,T445C,C3037T,A4926G,C6040T,C6286T,...",NaN,NaN,"1-35,29632-29903",NaN,16.0,0.0,0.0,307.0,...,2021-03-23T10:26:37,0,DEMIS-10640,Nordrhein-Westfalen,"Köln, Stadt",51109,DEMIS-10640,51109,"[{""lineage"": ""B.1.177.86"", ""method"": ""PANGOLIN...",2021-02
1229808,IMS-10640-CVDP-2F6EB4F4-0B25-4CE4-9EEC-0CAAE9B...,"C241T,C593T,C3037T,C4543T,C4999T,G5629T,T8503C...",NaN,NaN,"1-40,29819-29903",NaN,28.0,0.0,0.0,125.0,...,2021-03-23T10:26:37,0,DEMIS-10640,Nordrhein-Westfalen,"Köln, Stadt",51109,DEMIS-10640,51109,"[{""lineage"": ""B.1.160.28"", ""method"": ""PANGOLIN...",2021-02


In [24]:
merged_df = merged_df[(merged_df['date_of_sampling'] >= '2021-01-01') & (merged_df['date_of_sampling'] <= '2023-03-31')]

In [25]:
merged_df.to_csv("merged_dataset.csv", sep=";")